In [2]:
import skmob
import pandas as pd
import csv, os
from skmob.preprocessing import detection, clustering

device_id_list = []
for it in os.scandir('./staypoint'):
    if it.is_dir():
        if it.name.isdigit():
            device_id_list.append(it.name)
device_id_list.sort()

for device_id in device_id_list:
    stdf = skmob.TrajDataFrame.from_file('./staypoint/'+ device_id +'/stay_points.csv',
        latitude='latitude', longitude='longitude', user_id='deviceid', datetime='datetime')

    stdf['leaving_datetime'] = pd.to_datetime(stdf['leaving_datetime'])

    cstdf = clustering.cluster(stdf, cluster_radius_km=0.2, min_samples=1)

    stay_point_list = cstdf.values.tolist()
    cluster_list = list()

    for stay_point in stay_point_list:
        stay_point[3] = stay_point[3].strftime('%Y-%m-%d %H:%M:%S')
        stay_point[4] = stay_point[4].strftime('%Y-%m-%d %H:%M:%S')
        if stay_point[5] not in cluster_list:
            cluster_list.append(stay_point[5])
    cluster_list.sort()
    
    fields = ['deviceid', 'latitude', 'longitude','datetime', 'leaving_datetime', 'cluster']
    path = "./cluster/"+device_id   
    if not os.path.isdir(path):
        os.mkdir(path)
    with open("./cluster/"+device_id+'/clustering_result.csv', 'w', newline='') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(stay_point_list)
    
    for c in cluster_list:
        f = open("./cluster/"+device_id+"/cluster_"+str(c)+'.csv', 'w', newline='')
        write = csv.writer(f)
        write.writerow(fields)
        for stay_point in stay_point_list:
            if(stay_point[5]==c):
                write.writerow(stay_point)
        f.close()

In [3]:
stdf = skmob.TrajDataFrame.from_file('./staypoint/53/stay_points.csv',
    latitude='latitude', longitude='longitude', user_id='deviceid', datetime='datetime')

stdf['leaving_datetime'] = pd.to_datetime(stdf['leaving_datetime'])
m = stdf.plot_trajectory(max_users=1, start_end_markers=False)
stdf.plot_stops(max_users=1, map_f=m)


/Users/wonnx/Github/graduation/skmob/core/trajectorydataframe.py:569: UserWarning: If necessary, trajectories will be down-sampled to have at most `max_points` points. To avoid this, specify `max_points=None`.
  return plot.plot_trajectory(self, map_f=map_f, max_users=max_users, max_points=max_points, style_function=style_function,
